<a href="https://colab.research.google.com/github/supriyag123/PHD_Pub/blob/main/DGRNet%20Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import tensorflow as tf
import os
import math
import plotly.graph_objects as go
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, RepeatVector, TimeDistributed, Input
from keras.models import Model
from keras import backend as K
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import EarlyStopping
import keras.backend as K
from keras.callbacks import Callback
import plotly
import plotly.express as px # for data visualization
import seaborn as sns

generator_multiply = 5 #each input record will generate 100 random vectors from the latent space, given the mu and sigma generated by the encoder

#from keras.utils import plot_model
#import matplotlib.pyplot as plt

#window1 = np.load(r'/content/drive/MyDrive/PHD/2021/multivariate_long_sequences_WINDOW-500.npy')
#window2 = np.load(r'/content/drive/MyDrive/PHD/2021/multivariate_long_sequences_WINDOW-1000.npy')
#window = np.concatenate((window1, window2), axis=0)
#train_data = np.load(r'/content/drive/MyDrive/PHD/2021/multivariate_long_sequences-TRAIN.npy')
#test_data = np.load(r'/content/drive/MyDrive/PHD/2021/multivariate_long_sequences-TEST.npy')



#get data
train_data = np.load(r'/content/drive/MyDrive/PHD/2024/multivariate_long_sequences-TRAIN_hourly.npy')
index = 500
#We missed i=500 from processing the iosw. So here we are dropping row with index =500
train_data= np.delete(train_data, index, axis=0)

test_data = np.load(r'/content/drive/MyDrive/PHD/2024/multivariate_long_sequences-TEST_hourly.npy')
#all_data = np.concatenate((train_data,test_data),axis=0)
window_label = np.load(r'/content/drive/MyDrive/PHD/2024/multivariate_long_sequences_WINDOW-TRAIN_hourly.npy')
n_seq = train_data.shape[0]
window_size = train_data.shape[1]
n_features = train_data.shape[2]


plt.suptitle('Sub sequence plotting', fontsize='30')
plt.xlabel('Time', fontsize ='20')
plt.ylabel('Feature 1', fontsize='20')
plt.plot(train_data[:,:,1])
plt.show()

#---------------------------------Without VAE------------------------------------

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(train_data, window_label, test_size = 0.2, random_state = 42)

from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error


x_train = x_train.reshape((x_train.shape[0], x_train.shape[1]*n_features))
x_test = x_test.reshape((x_test.shape[0], x_test.shape[1]*n_features))

from sklearn.covariance import EllipticEnvelope
ee = EllipticEnvelope(contamination=0.05)
yhat = ee.fit_predict(x_train)
# select all rows that are not outliers
mask = yhat != -1
x_train, y_train = x_train[mask, :], y_train[mask]


x_train = x_train.reshape((x_train.shape[0], window_size, n_features))
x_test = x_test.reshape((x_test.shape[0], window_size, n_features))

from keras.layers import Dense, Activation
from keras.models import Sequential
##!pip uninstall tensorflow
#!pip install tensorflow==2.12.0
#from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import TransformedTargetRegressor
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import QuantileTransformer

model = Sequential()
model.add(LSTM(1000, input_shape=(x_train.shape[1],x_train.shape[2])))
model.add(Dense(1000, activation = 'relu'))
model.add(Dense(units = 512, activation = 'relu'))
model.add(Dense(units = 512, activation = 'relu'))
#model.add(Dense(units = 128, activation = 'relu'))
#model.add(Dense(units = 128, activation = 'relu'))
model.add(Dense(units = 1))
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_squared_error'])

#model.fit(x_train, y_train,epochs=5, batch_size=50, verbose=True)

#y_pred = model.predict(x_test)

#transform

model.fit(x_train, y_train,epochs=100, batch_size=50, verbose=True)
y_pred = model.predict(x_test)
score= r2_score(y_test,y_pred)
print("r2 score is ==",score)
print("mean_sqrd_error is==",mean_squared_error(y_test,y_pred))
print("root_mean_squared error of is==",np.sqrt(mean_squared_error(y_test,y_pred)))

#plt.scatter(y_test,y_pred);
#plt.xlabel('Actual');
#plt.ylabel('Predicted');
plt.plot(y_test[0:100], color = 'red', label = 'Real data')
plt.plot(y_pred[0:100], color = 'blue', label = 'Predicted data')
plt.title('Prediction')
plt.legend()
plt.show()

from sklearn.ensemble import IsolationForest
from sklearn.metrics import mean_absolute_error


np.savetxt(r'/content/drive/MyDrive/PHD/2024/MLPOutput/pred_2.csv',y_pred_t)
np.savetxt(r'/content/drive/MyDrive/PHD/2024/MLPOutput/real_2.csv',y_test)
print("MAE is==",mean_absolute_error(y_test,y_pred))

#########Isolation forest-------------------------#
iso = IsolationForest(contamination=0.1)
x_train, x_test, y_train, y_test = train_test_split(train_data, window_label, test_size = 0.2, random_state = 42)

x_train = x_train.reshape((x_train.shape[0], x_train.shape[1]*n_features))
x_test = x_test.reshape((x_test.shape[0], x_test.shape[1]*n_features))
yhat = iso.fit_predict(x_train)
mask = yhat != -1
x_train, y_train = x_train[mask, :], y_train[mask]
x_train = x_train.reshape((x_train.shape[0], window_size, n_features))
x_test = x_test.reshape((x_test.shape[0], window_size, n_features))

model = Sequential()
model.add(LSTM(1000, input_shape=(x_train.shape[1],x_train.shape[2])))
model.add(Dense(1000, activation = 'relu'))
model.add(Dense(units = 512, activation = 'relu'))
model.add(Dense(units = 512, activation = 'relu'))
#model.add(Dense(units = 128, activation = 'relu'))
#model.add(Dense(units = 128, activation = 'relu'))
model.add(Dense(units = 1))
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_squared_error'])

#model.fit(x_train, y_train,epochs=5, batch_size=50, verbose=True)

#y_pred = model.predict(x_test)

#transform

model.fit(x_train, y_train,epochs=100, batch_size=50, verbose=True)
y_pred = model.predict(x_test)
score= r2_score(y_test,y_pred)
print("r2 score is ==",score)
print("mean_sqrd_error is==",mean_squared_error(y_test,y_pred))
print("root_mean_squared error of is==",np.sqrt(mean_squared_error(y_test,y_pred)))

-----------------------------------------------------------------------------------

encoder = keras.models.load_model(r'/content/drive/MyDrive/PHD/2024/VAE_SIMULATION/vae-encoder-saved-round5-latent50-dim1000.model')
decoder = keras.models.load_model(r'/content/drive/MyDrive/PHD/2024/VAE_SIMULATION/vae-decoder-saved-round5-latent50-dim1000.model')

X_train_encoded = encoder.predict(train_data)
mu, logvar, z = X_train_encoded
sigma = tf.exp(0.5 * logvar)
batch = tf.shape(mu)[0]  #number of recors / batchs
dim = tf.shape(mu)[1] #Ndimension of latent variable
store = list()
storetemp = list()
#For each batch, iterate, get the generator_multipy number of latent vectors with same window_size

for i in range(0,batch):
  all_Z_i = tf.random.normal(shape=(generator_multiply,dim), mean = mu[i,:], stddev=sigma[i,:]) #all randorm vectors for this record i
  a = np.arange(generator_multiply)
  a.fill(window_label[i])
  c=np.concatenate(((all_Z_i,a[:,None])),axis=1)
  store.append(c)

results=np.concatenate(store,axis=0)
np.save(r'/content/drive/MyDrive/PHD/2024/labelled_subsquence_data',results)
#Regression fitting
x=results[:,:-1]
y=results[:,dim]
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 42)


#################  ANN to do regression###########################
#######  https://stackoverflow.com/questions/49008074/how-to-create-a-neural-network-for-regression  #######################
###########  Nh = Ns/(α∗ (Ni + No)) ##########################

from keras.layers import Dense, Activation
from keras.models import Sequential
##!pip uninstall tensorflow
#!pip install tensorflow==2.12.0
#from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import TransformedTargetRegressor
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import QuantileTransformer


transformer = StandardScaler()

y_train_transformed = transformer.fit_transform(y_train.reshape(-1,1)).flatten()
y_test_transformed = transformer.fit_transform(y_test.reshape(-1,1)).flatten()

model = Sequential()
model.add(Dense(512, activation = 'relu', input_dim = dim))
model.add(Dense(units = 256, activation = 'relu'))
model.add(Dense(units = 256, activation = 'relu'))
model.add(Dense(units = 128, activation = 'relu'))
model.add(Dense(units = 128, activation = 'relu'))
model.add(Dense(units = 1))
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_squared_error'])

#model.fit(x_train, y_train,epochs=5, batch_size=50, verbose=True)

#y_pred = model.predict(x_test)

#transform

model.fit(x_train, y_train_transformed,epochs=500, batch_size=50, verbose=True)
y_pred = model.predict(x_test)
y_pred_t = transformer.inverse_transform(y_pred.reshape(-1,1)).flatten()
score= r2_score(y_test,y_pred_t)
print("r2 score is ==",score)
print("mean_sqrd_error is==",mean_squared_error(y_test,y_pred_t))
print("root_mean_squared error of is==",np.sqrt(mean_squared_error(y_test,y_pred_t)))

#plt.scatter(y_test,y_pred);
#plt.xlabel('Actual');
#plt.ylabel('Predicted');
plt.plot(y_test[0:100], color = 'red', label = 'Real data')
plt.plot(y_pred_t[0:100], color = 'blue', label = 'Predicted data')
plt.title('Prediction')
plt.legend()
plt.show()


np.savetxt(r'/content/drive/MyDrive/PHD/2024/MLPOutput/pred_2.csv',y_pred_t)
np.savetxt(r'/content/drive/MyDrive/PHD/2024/MLPOutput/real_2.csv',y_test)
#----------------Random forest ----------------------------------
###### Random forrest ergression
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.compose import TransformedTargetRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import PowerTransformer
from sklearn.linear_model import HuberRegressor
 # create regressor object
#regressor = RandomForestRegressor(n_estimators = 50, random_state = 0)
# prepare the model with input scaling
pipeline = Pipeline(steps=[('power', PowerTransformer()), ('model',RandomForestRegressor(n_estimators = 100, random_state = 0))])
#pipeline = Pipeline(steps=[('power', PowerTransformer()), ('model',HuberRegressor())])

# prepare the model with target scaling
model = TransformedTargetRegressor(regressor=pipeline, transformer=PowerTransformer())

# fit the regressor with x and y data
pipeline.fit(x_train,y_train)
y_pred = pipeline.predict(x_test)  # test the output by changing values
# creating an object of LinearRegression class
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
# predicting the accuracy score

score=r2_score(y_test,y_pred)
print("r2 score is ==",score)
print("mean_sqrd_error is==",mean_squared_error(y_test,y_pred))
print("root_mean_squared error of is==",np.sqrt(mean_squared_error(y_test,y_pred)))
plt.scatter(y_test,y_pred);
plt.xlabel('Actual');
plt.ylabel('Predicted');


plt.plot(y_test[5000:5049], color = 'red', label = 'Real data')
plt.plot(y_pred_t[5000:5049], color = 'blue', label = 'Predicted data')
plt.title('Prediction')
plt.legend()
plt.show()




Epoch 1/100
447/447 [==============================] - 38s 81ms/step - loss: 1387.5629 - mean_squared_error: 1387.5629
Epoch 2/100
447/447 [==============================] - 35s 77ms/step - loss: 1247.8260 - mean_squared_error: 1247.8260
Epoch 3/100
447/447 [==============================] - 36s 80ms/step - loss: 898.0354 - mean_squared_error: 898.0354
Epoch 4/100
447/447 [==============================] - 35s 79ms/step - loss: 800.9507 - mean_squared_error: 800.9507
Epoch 5/100
447/447 [==============================] - 36s 80ms/step - loss: 744.1084 - mean_squared_error: 744.1084
Epoch 6/100
447/447 [==============================] - 36s 80ms/step - loss: 717.5817 - mean_squared_error: 717.5817
Epoch 7/100
447/447 [==============================] - 36s 80ms/step - loss: 700.5239 - mean_squared_error: 700.5239
Epoch 8/100
447/447 [==============================] - 36s 81ms/step - loss: 693.1699 - mean_squared_error: 693.1699
Epoch 9/100
447/447 [==============================] - 36s 8